In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate



# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter


# Initialize Dash application using JupyterDash to run within a Jupyter environment.
app = JupyterDash('SimpleExample')

# Set up styles for the dashboard components for a clean layout.
graph_style = {'width': '48%', 'display': 'inline-block'}
data_table_style = {'width': '100%', 'display': 'inline-block'}
map_style = {'height': '500px', 'width': '48%'}
image_filename = 'Animal.png'  # Image file used in the dashboard

# Encode the image for use in HTML.
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Define your connection parameters to the database.
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32486, 'AAC', 'animals')

# Retrieve initial data from the database and prepare it for use in the app.
df = pd.DataFrame.from_records(shelter.read({}))
if 'health' in df.columns:
    df['health'] = df['health'].astype(str)
df.drop(columns=['_id'], inplace=True, errors='ignore')  # Remove the '_id' column to avoid issues with DataTable.

# Set up the layout of the application with various components.
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '200px'})),
    html.Center(html.B(html.H1('Animal Shelter Dashboard - Mohamed Jaddour'))),
    html.Hr(),
    dcc.RadioItems(
        id='rescue-type-selector',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness', 'value': 'Mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset', 
        labelStyle={'display': 'inline-block'}
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        sort_action='native',
        page_action='native',
        page_size=10,
        row_selectable='single',
        selected_rows=[0]
    ),
    html.Div([
        dcc.Graph(id='pie-chart-id', style=graph_style),
        html.Div(id='map-id', style=map_style,
            children=[dl.Map(style={'width': '100%', 'height': '100%'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(id='base-layer-id')])]
        )
    ], style={'display': 'flex', 'flex-wrap': 'wrap'})
])

# Callback to update data table based on the rescue type selected by the user.
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-selector', 'value')]
)
def filter_data(rescue_type):
    # Fetch the filtered data from the database using RESCUE_CRITERIA
    if rescue_type == 'Reset':
        df = pd.DataFrame.from_records(shelter.read({}))
    else:
        criteria = RESCUE_CRITERIA[rescue_type]
        # Build the query based on the criteria
        query = {
              '$and':[
                 {'breed': {'$in': criteria['breeds']}},
                 {'sex_upon_outcome': criteria['sex']},
                {'age_upon_outcome_in_weeks': {'$gte': criteria['age_weeks'][0], '$lte': criteria['age_weeks'][1]}}
            ] 
        }
        df = pd.DataFrame.from_records(shelter.read(query))
        
    if 'health' in df.columns:
        df['health'] = df['health'].astype(str).fillna('Unknown')
        
    df.drop(columns=['_id'], inplace=True, errors='ignore')
    return df.to_dict('records')

# Callback to generate a pie chart of breed distribution based on the current data table.
@app.callback(    
    Output('pie-chart-id', 'figure'),
     [Input('datatable-id', 'data')
      ]
)
def update_pie_chart(rows):
    if not rows:
        raise PreventUpdate
    
    dff = pd.DataFrame(rows)
    
    if 'breed' not in dff.columns:
        raise PreventUpdate
       
    fig = px.pie(dff, names='breed', title=f"Distrubution of Breeds")
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(
        height=600,
        width=600,
        margin=dict(l=40, r=40, t=40, b=40)
    )
    return fig


# Callback to update the map based on the selected row in the data table.
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'), Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if not viewData or not index:
        raise PreventUpdate
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    latitude = dff.iloc[row, 13]
    longitude = dff.iloc[row, 14]
    breed = dff.iloc[row, 4]
    animal_name = dff.iloc[row, 9]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H1('Animal Name'), html.P(animal_name)])
            ])
        ])
    ]

# Define criteria for filtering data based on rescue type.
RESCUE_CRITERIA = {
    'Water': {'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'], 'sex': 'Intact Female', 'age_weeks': (26, 156)},
    'Mountain': {'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'], 'sex': 'Intact Male', 'age_weeks': (26, 156)},
    'Disaster': {'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'], 'sex': 'Intact Male', 'age_weeks': (20, 300)},
    'Reset': {}
}

# Run the server with debug mode to enable live updates during development.
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8012/
